# 3D medical image segmentation with one slice annotation

This notebook shows how to use SAM 2 for interactive segmentation in 3D images. It will cover the following:

- propagating a mask to get _masklets_ throughout the video

In [1]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
# use bfloat16 for the entire notebook
torch.autocast(device_type="cuda", dtype=torch.bfloat16).__enter__()

if torch.cuda.get_device_properties(0).major >= 8:
    # turn on tfloat32 for Ampere GPUs (https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices)
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

### Loading the SAM 2 video predictor

In [3]:
from sam2.build_sam import build_sam2_video_predictor

sam2_checkpoint = "../checkpoints/sam2_finetuned_small.pth"
model_cfg = "configs/sam2/sam2_hiera_s.yaml"
#sam2_checkpoint = "../checkpoints/sam2_hiera_large.pt"
#model_cfg = "sam2_hiera_l.yaml"

predictor = build_sam2_video_predictor(model_cfg, sam2_checkpoint)

In [4]:
# def show_mask(mask, ax, obj_id=None, random_color=False):
#     if random_color:
#         color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
#     else:
#         cmap = plt.get_cmap("tab10")
#         cmap_idx = 0 if obj_id is None else obj_id
#         color = np.array([*cmap(cmap_idx)[:3], 0.6])
#     h, w = mask.shape[-2:]
#     mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
#     ax.imshow(mask_image)

# Color masks Red Green and Blue
def show_mask(mask, ax, random_color=False):
    # Define colors for each class (RGBA for transparency)
    colors = {
        0: np.array([0.0, 0.0, 0.0, 0.0]),  # Transparent background
        1: np.array([1.0, 0.0, 0.0, 0.6]),  # Red for class 1
        2: np.array([0.0, 1.0, 0.0, 0.6]),  # Green for class 2
        3: np.array([0.0, 0.0, 1.0, 0.6])   # Blue for class 3
    }

    # Random color option
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
        mask_image = mask.reshape(mask.shape[0], mask.shape[1], 1) * color.reshape(1, 1, -1)
    else:
        # Initialize an empty image with the same shape as the mask but with RGBA channels
        h, w = mask.shape[-2:]
        mask_image = np.zeros((h, w, 4))

        # Apply color based on class value
        for class_id, color in colors.items():
            mask_image += (mask == class_id).reshape(h, w, 1) * color.reshape(1, 1, -1)

    ax.imshow(mask_image)
    ax.axis("off")


def show_points(coords, labels, ax, marker_size=200):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)

In [5]:
# Initialize a global counter to track the color cycle
color_cycle = [np.array([1.0, 0.0, 0.0, 0.6]),  # Red
               np.array([0.0, 1.0, 0.0, 0.6]),  # Green
               np.array([0.0, 0.0, 1.0, 0.6])]  # Blue

color_index = 0  # Start with red

def show_mask_sam(mask, ax, random_color=False):
    global color_index  # Use the global color_index to track color

    # Get the current color from the cycle
    color = color_cycle[color_index]

    # Random color option (if needed)
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
        mask_image = mask.reshape(mask.shape[0], mask.shape[1], 1) * color.reshape(1, 1, -1)
    else:
        # Initialize an empty image with the same shape as the mask but with RGBA channels
        h, w = mask.shape[-2:]
        mask_image = np.zeros((h, w, 4))

        # Apply color based on the mask class
        mask_image += (mask == 1).reshape(h, w, 1) * color.reshape(1, 1, -1)

    # Show the mask
    ax.imshow(mask_image)
    ax.axis("off")

    # Update the color cycle (move to the next color)
    color_index = (color_index + 1) % len(color_cycle)


SAM 2 requires stateful inference for interactive video segmentation, so we need to initialize an **inference state** on this video.

During initialization, it loads all the JPEG frames in `video_path` and stores their pixels in `inference_state` (as shown in the progress bar below).
### Propagate the prompts to get the masklet across the video
#### Calculate Dice and IoU for all test and write csv

In [6]:
import csv
import torch
from monai.metrics import DiceHelper, compute_surface_dice

# Define paths to image and mask directories
#video_dir = "./videos/Test/"
video_dir = "./videos/MedicalSAM2_Test"
images_dir = os.path.join(video_dir, "image")
masks_dir = os.path.join(video_dir, "mask")

#middle_masks_dir = "./sam_output_middle/"
middle_masks_dir = "./MnM_output_middle/"

# Initialize an empty list to collect scores
results = []

# Iterate through each patient folder in the images directory
for patient_folder in os.listdir(images_dir):
    patient_images_path = os.path.join(images_dir, patient_folder)
    patient_masks_path = os.path.join(masks_dir, patient_folder)
    middle_patient_masks_path = os.path.join(middle_masks_dir, patient_folder)
    
    # Get all JPEG frame names in the patient's images folder
    frame_names = [
        p for p in os.listdir(patient_images_path)
        if os.path.splitext(p)[-1] in [".jpg", ".jpeg", ".JPG", ".JPEG"]
    ]
    frame_names.sort(key=lambda p: int(os.path.splitext(p)[0]))  # Sort by frame number
    frame_idx = len(frame_names) // 2
   
    # Mask Ground truth - middle slice
    print(middle_patient_masks_path)
    out_mask_logits = np.load(os.path.join(middle_patient_masks_path, frame_names[frame_idx].replace(".jpg", ".npy")))
    
    # run propagation to get the mask for the entire video
    inference_state = predictor.init_state(video_path=patient_images_path)

    ann_obj_id = 1
    
    # Initialize dictionaries to store video segments for each class
    class_1_segments = {}
    class_2_segments = {}
    class_3_segments = {}
    
    for class_id in [1, 2, 3]:
        
        # Need to reset state for each class
        predictor.reset_state(inference_state)
        
        # Add new mask for the current class
        predictor.add_new_mask(inference_state=inference_state, frame_idx=frame_idx, obj_id=ann_obj_id, mask=out_mask_logits == class_id)
        
        # Initialize video_segments for this class
        video_segments = {}  # video_segments contains the per-frame segmentation results
    
        # Direct propagation   
        for out_frame_idx, out_obj_ids, class_mask in predictor.propagate_in_video(inference_state):
            # Store results for the current class in the appropriate variable
            for i, out_obj_id in enumerate(out_obj_ids):
                class_mask_np = (class_mask[i] > 0.0).cpu().numpy()
                if class_id == 1:
                    class_1_segments[out_frame_idx] = {out_obj_id: class_mask_np}
                elif class_id == 2:
                    class_2_segments[out_frame_idx] = {out_obj_id: class_mask_np}
                elif class_id == 3:
                    class_3_segments[out_frame_idx] = {out_obj_id: class_mask_np}
    
        # Reverse propagation    
        for out_frame_idx, out_obj_ids, class_mask in predictor.propagate_in_video(inference_state, start_frame_idx=frame_idx-1, reverse=True):
            # Store results for the current class in the appropriate variable
            for i, out_obj_id in enumerate(out_obj_ids):
                class_mask_np = (class_mask[i] > 0.0).cpu().numpy()
                if class_id == 1:
                    class_1_segments[out_frame_idx] = {out_obj_id: class_mask_np}
                elif class_id == 2:
                    class_2_segments[out_frame_idx] = {out_obj_id: class_mask_np}
                elif class_id == 3:
                    class_3_segments[out_frame_idx] = {out_obj_id: class_mask_np}
    
    # Class LV
    video_seg_3d_1 = np.stack([class_1_segments[k][1] for k in class_1_segments])
    video_seg_3d_2 = np.stack([class_2_segments[k][1] for k in class_2_segments])
    video_seg_3d_3 = np.stack([class_3_segments[k][1] for k in class_3_segments])
    #gt_3d = np.stack([np.load(os.path.join(video_dir, frame_names[k].replace(".jpg", ".npy")))[None] for k in class_1_segments])
    
    # Initialize empty lists to hold the 3D arrays for each class (0, 1, 2, 3)
    gt_3d_class_0 = []
    gt_3d_class_1 = []
    gt_3d_class_2 = []
    gt_3d_class_3 = []
    
    # Iterate through class_1_segments to load the .npy files and process them
    for k in class_1_segments:
        # Load the .npy file corresponding to frame_names[k]
        frame_path = os.path.join(patient_masks_path, frame_names[k].replace(".jpg", ".npy"))
        mask = np.load(frame_path)
        
        # Create separate binary masks for each class (0, 1, 2, 3)
        mask_class_0 = (mask == 0).astype(np.uint8)  # 1 for class 0, 0 otherwise
        mask_class_1 = (mask == 1).astype(np.uint8)  # 1 for class 1, 0 otherwise
        mask_class_2 = (mask == 2).astype(np.uint8)  # 1 for class 2, 0 otherwise
        mask_class_3 = (mask == 3).astype(np.uint8)  # 1 for class 3, 0 otherwise
        
        # Append each binary mask (with new axis) to the corresponding list
        gt_3d_class_0.append(mask_class_0[None])  # Shape will be (1, height, width)
        gt_3d_class_1.append(mask_class_1[None])
        gt_3d_class_2.append(mask_class_2[None])
        gt_3d_class_3.append(mask_class_3[None])
    
    # Stack the lists into 3D arrays
    gt_3d_class_0 = np.stack(gt_3d_class_0)  # Shape: (num_frames, 1, height, width)
    gt_3d_class_1 = np.stack(gt_3d_class_1)
    gt_3d_class_2 = np.stack(gt_3d_class_2)
    gt_3d_class_3 = np.stack(gt_3d_class_3)
    
    # Class RV
    n_classes, batch_size = 1, 1
    
    spatial_shape = (video_seg_3d_1.shape[0], video_seg_3d_1.shape[2], video_seg_3d_1.shape[3]) 
    y_pred = torch.tensor(video_seg_3d_1).float().reshape(batch_size, n_classes, *spatial_shape)  # prediction
    y = torch.tensor(gt_3d_class_1).float().reshape(batch_size, n_classes, *spatial_shape)  # ground truth
    
    score, not_nans = DiceHelper(include_background=False, sigmoid=True, softmax=True)(y_pred, y)
    #print(f"Patient folder: {os.path.basename(patient_images_path)}, RV dice score: {score}")
    rv_score = score.item()  # Convert to Python float if it's a tensor
    
    # Class MYO
    spatial_shape = (video_seg_3d_2.shape[0], video_seg_3d_2.shape[2], video_seg_3d_2.shape[3]) 
    y_pred = torch.tensor(video_seg_3d_2).float().reshape(batch_size, n_classes, *spatial_shape)  # prediction
    y = torch.tensor(gt_3d_class_2).float().reshape(batch_size, n_classes, *spatial_shape)  # ground truth
    
    score, not_nans = DiceHelper(include_background=False, sigmoid=True, softmax=True)(y_pred, y)
    #print('MYO dice score:', score)
    myo_score = score.item()
    
    # Class LV
    spatial_shape = (video_seg_3d_3.shape[0], video_seg_3d_3.shape[2], video_seg_3d_3.shape[3]) 
    y_pred = torch.tensor(video_seg_3d_3).float().reshape(batch_size, n_classes, *spatial_shape)  # prediction
    y = torch.tensor(gt_3d_class_3).float().reshape(batch_size, n_classes, *spatial_shape)  # ground truth
    
    score, not_nans = DiceHelper(include_background=False, sigmoid=True, softmax=True)(y_pred, y)
    #print('LV dice score:', score)
    lv_score = score.item()

    # Append results for this patient
    results.append([os.path.basename(patient_images_path), rv_score, myo_score, lv_score])


# Write all results to CSV
csv_path = "dice_sam_input_sam2_mnm_finetuned_small.csv"
with open(csv_path, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Patient Folder", "RV Dice Score", "MYO Dice Score", "LV Dice Score"])
    writer.writerows(results)

print(f"Dice scores saved to {csv_path}")


./MnM_output_middle/A1K2P5_ED


propagate in video:   0%|                                                                        | 0/5 [00:00<?, ?it/s]C:\Users\Windows\Desktop\Tijana\sam2-main\sam2-main\sam2\modeling\sam\transformer.py:344: UserWarning: Memory efficient kernel not used because: (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:778.)
  out = F.scaled_dot_product_attention(q, k, v, dropout_p=dropout_p)
C:\Users\Windows\Desktop\Tijana\sam2-main\sam2-main\sam2\modeling\sam\transformer.py:344: UserWarning: Memory Efficient attention has been runtime disabled. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen/native/transformers/sdp_utils_cpp.h:558.)
  out = F.scaled_dot_product_attention(q, k, v, dropout_p=dropout_p)
C:\Users\Windows\Desktop\Tijana\sam2-main\sam2-main\sam2\modeling\sam\transformer.py:344: UserWarning: Flash attention kernel not used because: (Tri

./MnM_output_middle/A1K2P5_ES


propagate in video: 100%|████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 22.28it/s]


./MnM_output_middle/A2H5K9_ED


...

./MnM_output_middle/Y6Y9Z2_ED


propagate in video: 100%|████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 21.62it/s]


./MnM_output_middle/Y6Y9Z2_ES


propagate in video: 100%|████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 22.23it/s]


Dice scores saved to dice_sam_input_sam2_mnm_finetuned_small.csv
